In [1]:
import os, pandas as pd, torch, numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
train_window = 100
epochs = 150
scaler = MinMaxScaler(feature_range = (0, 1))

In [3]:
df_train = pd.read_csv("train.csv", engine = "python", encoding = "utf-8")

In [4]:
data_train = scaler.fit_transform(df_train.values)
x_train, y_train =\
    [ row[:2] for row in data_train ], [ row[2:] for row in data_train ]

In [5]:
train_input = []
for idx in range(len(x_train) - train_window - 1):
    xi = torch.Tensor(np.array(x_train[idx:idx + train_window]))
    yi = torch.Tensor(np.array(y_train[idx + train_window + 1]))

    train_input.append(( xi, yi ))

In [6]:
class CustomModel(torch.nn.Module):
    def __init__(self, n_input = 1, n_output = 1, n_layer = 1, n_hidden_layer = 100, nonlinearity = "tanh"):
        super().__init__()

        self.rnn = torch.nn.RNN(n_input, n_hidden_layer, num_layers = n_layer, nonlinearity = nonlinearity)
        self.linear = torch.nn.Linear(n_hidden_layer, n_output)

    def forward(self, x:np.ndarray):
        xt = torch.Tensor(x)
        out, _ = self.rnn(xt.view(len(xt), 1, 2))
        return self.linear(out.view(len(xt), -1))[-1]

In [7]:
model = CustomModel(n_input = 2, n_layer = 2, n_hidden_layer = 150)
loss_func = torch.nn.MSELoss()
opt = torch.optim.Adam(model.parameters(), lr = 0.001)

In [8]:
for step in range(epochs):
    for xi, yi in train_input:
        opt.zero_grad()

        pred = model(xi)

        loss = loss_func(pred, yi)

        opt.zero_grad()
        loss.backward()
        opt.step()

model.eval()

CustomModel(
  (rnn): RNN(2, 150, num_layers=2)
  (linear): Linear(in_features=150, out_features=1, bias=True)
)

In [25]:
torch.save(model, os.path.join("model", "custom.pth"))

In [9]:
df_predict = pd.read_csv("predict.csv", engine = "python", encoding = "utf-8")

In [10]:
data_predict = scaler.fit_transform(df_train.values)
x_predict = [ row[:2] for row in data_predict ]

In [14]:
y_predict = []
for xi in x_predict:
    with torch.no_grad():
        y_predict.append(model(np.array([xi])).item())

In [15]:
y_predict

[0.3656957745552063,
 0.3662746846675873,
 0.3676449954509735,
 0.3670092523097992,
 0.3631322979927063,
 0.37164777517318726,
 0.3647852838039398,
 0.36869168281555176,
 0.36491692066192627,
 0.36857545375823975,
 0.3710847795009613,
 0.36937254667282104,
 0.3656628131866455,
 0.3658861517906189,
 0.37084364891052246,
 0.36967629194259644,
 0.36906713247299194,
 0.3690148890018463,
 0.3644687533378601,
 0.36751818656921387,
 0.36930787563323975,
 0.3670324981212616,
 0.3719156086444855,
 0.3703067898750305,
 0.36976826190948486,
 0.36563485860824585,
 0.3677990436553955,
 0.37176159024238586,
 0.3641509413719177,
 0.37138813734054565,
 0.36739760637283325,
 0.36630067229270935,
 0.36519530415534973,
 0.36775481700897217,
 0.36718320846557617,
 0.36615002155303955,
 0.3721664547920227,
 0.36788690090179443,
 0.36663731932640076,
 0.36954495310783386,
 0.36924684047698975,
 0.36485663056373596,
 0.3660263419151306,
 0.3690778613090515,
 0.37098515033721924,
 0.36924487352371216,
 0.3660

In [20]:
pred_result = scaler.inverse_transform(np.array([
    xi.tolist() + [ yi ]
    for xi, yi in zip(x_predict, y_predict)
]))

In [21]:
pred_result

array([[30.78521947, 24.92250314, 28.48444202],
       [15.79655555, 19.62462122, 28.50524869],
       [23.98078111, 26.36733004, 28.55449918],
       ...,
       [26.3732961 , 34.56031429, 28.65805481],
       [21.12495383, 27.79414257, 28.59732397],
       [30.12029884, 28.25609519, 28.53819339]])

In [22]:
df_predict["YPredict"] = [
    item[2]
    for item in pred_result
]

In [23]:
df_predict.head()

,X1,X2,Y,YPredict
0,22.418648,28.355352,37.266023,28.484442
1,23.886250,26.821878,23.061118,28.505249
2,23.056293,23.648298,26.677375,28.554499
3,11.322328,19.561843,38.657209,28.531650
4,31.550013,20.258503,40.231940,28.392308
